In [56]:
import pandas as pd

# Load the FAQ data
documents = pd.read_csv('faqs.csv')
documents = documents.drop_duplicates(subset=['Category', 'Answer', 'Category'], keep='first')
    
documents.head()

,Category,Question,Answer
0,--Bonitätsprüfung,Wie kann die Bonität nachgewiesen werden?,Zweck einer Bonitätsprüfung (siehe https://vd...
1,--Profi-Online,Darf das Konsortium nur aus Forschungseinricht...,Bitte entnehmen Sie die Information zu den An...
2,--Profi-Online,Können mehrere Personen Rechte für die Nutzerv...,Es können mehrere Personen eines Zuwendungsem...
3,--Profi-Online,Sollte die Projektleitung einen profi-Online Z...,Ob die Projektleitenden einen Zugang zu profi...
4,--Profi-Online,"Wer schaltet die Zugänge für ""profi-Online"" frei?",Der Antrag auf profi-online ist der VDIVDE IT...


In [77]:
import hashlib

def generate_document_id(doc):
    combined = f"{doc['Category']}-{doc['Question']}-{doc['Answer'][:10]}-{doc['Answer'][:-10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [78]:
def generate_document_ids_for_dataframe(df):
    df['id'] = df.apply(generate_document_id, axis=1)
    return df

In [79]:
documents = generate_document_ids_for_dataframe(documents)

In [80]:
documents

,Category,Question,Answer,id
0,--Bonitätsprüfung,Wie kann die Bonität nachgewiesen werden?,Zweck einer Bonitätsprüfung (siehe https://vd...,bead1965
1,--Profi-Online,Darf das Konsortium nur aus Forschungseinricht...,Bitte entnehmen Sie die Information zu den An...,99541b90
2,--Profi-Online,Können mehrere Personen Rechte für die Nutzerv...,Es können mehrere Personen eines Zuwendungsem...,84ae26c1
3,--Profi-Online,Sollte die Projektleitung einen profi-Online Z...,Ob die Projektleitenden einen Zugang zu profi...,0817bd44
4,--Profi-Online,"Wer schaltet die Zugänge für ""profi-Online"" frei?",Der Antrag auf profi-online ist der VDIVDE IT...,f06cd838
...,...,...,...,...
1003,---Allgemeine Informationen,Welche anderen Dokumente können (ggf. alternat...,Für die Vorlage des Verwendungsnachweises sin...,d1404867
1013,---Allgemeine Informationen,Wie wird der Zeitnachweis für das Personal gef...,Für Hochschulen und andere grundfinanzierte E...,15def961
1014,---Allgemeine Informationen,Wie wird die Jahressonderzahlung bei Mitarbeit...,Die Jahressonderzahlung darf nur anteilig für...,23732604
1017,---Allgemeine Informationen,Wo finde ich das Formular für den Abschlussber...,Für den Sachbericht gibt es kein allgemein gü...,02cf6410


In [81]:
from collections import defaultdict

hashes = defaultdict(list)

for _, doc in documents.iterrows():
    doc_id = doc['id']
    hashes[doc_id].append(doc)


In [82]:
len(hashes), len(documents)


(833, 833)

In [83]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [84]:
hashes["06f888e6"]

[]

In [85]:
documents[documents["id"]=="8f1f4597"]

,Category,Question,Answer,id


In [99]:
prompt_template = """
Du emulierst einen Studenten, der an unserem Kurs teilnimmt.
Formuliere 5 Fragen, die dieser Student auf Basis eines FAQ-Eintrags stellen könnte. Der Eintrag
sollte die Antwort auf die Fragen enthalten, und die Fragen sollten vollständig und nicht zu kurz sein.
Verwende, wenn möglich, so wenige Wörter wie möglich aus dem Eintrag.

Der Eintrag:

Kategorie: {Category}
Frage: {Question}
Antwort: {Answer}

Gib das Ergebnis als parsierbares JSON ohne Codeblöcke aus:

["frage1", "frage2", ..., "frage5"]
""".strip()


In [100]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


from openai import OpenAI
client = OpenAI()

# Load your OpenAI API key
client.api_key = os.getenv("OPENAI_API_KEY")

In [101]:

def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [102]:
from tqdm.auto import tqdm


In [103]:
results = {}


In [104]:
from tqdm import tqdm

for _, doc in tqdm(documents.iterrows(), total=len(documents)): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions


100%|██████████| 833/833 [29:57<00:00,  2.16s/it] 


In [105]:
import pickle

# Assuming 'results' is the dictionary containing your generated questions
# Replace 'results' with your actual variable

file_path = 'generated_questions.bin'

with open(file_path, 'wb') as f:
    pickle.dump(results, f)

print(f'Results saved to {file_path}')


Results saved to generated_questions.bin


In [106]:
with open('generated_questions.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [108]:
import json
parsed_results = {}

for doc_id, json_questions in results.items():
    parsed_results[doc_id] = json.loads(json_questions)

In [111]:
doc_index = {doc['id']: doc for _, doc in documents.iterrows()}
final_results = []

for doc_id, questions in parsed_results.items():
    category = doc_index[doc_id]['Category']
    for q in questions:
        final_results.append((q, category, doc_id))

In [112]:
import pandas as pd


In [113]:
df = pd.DataFrame(final_results, columns=['Question', 'Category', 'Document'])


In [114]:
df.head()

,Question,Category,Document
0,Welche Unterlagen müssen zur Bonitätsprüfung e...,--Bonitätsprüfung,bead1965
1,Wer wird zur Bonitätsprüfung aufgefordert?,--Bonitätsprüfung,bead1965
2,"Was passiert, wenn kein geprüfter Jahresabschl...",--Bonitätsprüfung,bead1965
3,Was ist der Zweck einer Bonitätsprüfung?,--Bonitätsprüfung,bead1965
4,Wo finde ich weitere Informationen zur Bonität...,--Bonitätsprüfung,bead1965


In [115]:
df.to_csv('ground-truth-data.csv', index=False)
